In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jaccard
from Levenshtein import distance as edit_distance
from pyjarowinkler.distance import get_jaro_distance
from nltk.util import ngrams
from collections import defaultdict
from tqdm import tqdm

In [2]:
%pip install --upgrade --force-reinstall python-Levenshtein


  Using cached python_Levenshtein-0.26.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached levenshtein-0.26.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.2 kB)
  Using cached rapidfuzz-3.10.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
Using cached python_Levenshtein-0.26.1-py3-none-any.whl (9.4 kB)
Using cached levenshtein-0.26.1-cp312-cp312-macosx_11_0_arm64.whl (157 kB)
Using cached rapidfuzz-3.10.1-cp312-cp312-macosx_11_0_arm64.whl (1.4 MB)
  Attempting uninstall: rapidfuzz
    Found existing installation: RapidFuzz 3.10.1
    Uninstalling RapidFuzz-3.10.1:
      Successfully uninstalled RapidFuzz-3.10.1
  Attempting uninstall: Levenshtein
    Found existing installation: Levenshtein 0.26.1
    Uninstalling Levenshtein-0.26.1:
      Successfully uninstalled Levenshtein-0.26.1
  Attempting uninstall: python-Levenshtein
    Found existing installation: python-Levenshtein 0.26.1
    Uninstalling python-Levenshtein-0.26.1:
      Successfully uninstalled python-Levenshtein-0.26

In [3]:
%pip install pyjarowinkler


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
data = pd.read_csv("building_permits.csv")
data

/var/folders/t8/c3gz51w107s3mw7rbv_100lr0000gn/T/ipykernel_14541/2549140864.py:1: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("building_permits.csv")


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198895,M862628,8,otc alterations permit,12/05/2017,0113,017A,1228,NaN,Montgomery,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489337276729
198896,201712055595,8,otc alterations permit,12/05/2017,0271,014,580,NaN,Bush,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,NaN,NaN,1489462354993
198897,M863507,8,otc alterations permit,12/06/2017,4318,019,1568,NaN,Indiana,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489539379952
198898,M863747,8,otc alterations permit,12/06/2017,0298,029,795,NaN,Sutter,St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489608233656


In [3]:
df = pd.DataFrame(data)

# Updated Clean Text Function
def clean_text(text):
    if pd.isna(text):  # Handle NaN
        return ""
    text = str(text)  # Convert non-string values to string
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    return text.strip()

# Apply the cleaning function
df["cleaned_description"] = df["Description"].apply(clean_text)

# View the cleaned DataFrame
print(df)

       Permit Number  Permit Type            Permit Type Definition  \
0       201505065519            4                      sign - erect   
1       201604195146            4                      sign - erect   
2       201605278609            3  additions alterations or repairs   
3       201611072166            8            otc alterations permit   
4       201611283529            6                       demolitions   
...              ...          ...                               ...   
198895       M862628            8            otc alterations permit   
198896  201712055595            8            otc alterations permit   
198897       M863507            8            otc alterations permit   
198898       M863747            8            otc alterations permit   
198899       M864287            8            otc alterations permit   

       Permit Creation Date Block   Lot  Street Number Street Number Suffix  \
0                05/06/2015  0326   023            140              

In [4]:


# 2. String-Based Distance Functions
def compute_edit_distance(s1, s2):
    """Calculate edit distance."""
    return edit_distance(s1, s2)

def compute_jaro_distance(s1, s2):
    """Calculate Jaro-Winkler distance."""
    return get_jaro_distance(s1, s2, winkler=True, scaling=0.1)

# 3. Item-Based Distance Functions
#def compute_tfidf_similarity(texts):
#    """Calculate TF-IDF Cosine Similarity."""
#   vectorizer = TfidfVectorizer()
#   tfidf_matrix = vectorizer.fit_transform(texts)
#    cosine_sim = cosine_similarity(tfidf_matrix)
#   return cosine_sim

def compute_tfidf_similarity(texts, max_features=1000, batch_size=500):
    """
    Calculate TF-IDF Cosine Similarity with memory optimizations.
    Args:
        texts (list of str): The cleaned text data.
        max_features (int): Maximum number of features for TF-IDF vectorization.
        batch_size (int): Number of rows to process in each batch.
    Returns:
        generator: Sparse similarity matrices for each batch.
    """
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(texts)

    # Batch processing for cosine similarity
    n = tfidf_matrix.shape[0]
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        batch_matrix = tfidf_matrix[start:end]
        # Sparse cosine similarity
        sim_matrix = cosine_similarity(batch_matrix, tfidf_matrix, dense_output=False)
        yield sim_matrix

def compute_jaccard_distance(s1, s2):
    """Calculate Jaccard distance."""
    set1 = set(s1.split())
    set2 = set(s2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return 1 - intersection / union



In [5]:
# 4. Process Distances
# Example: Calculate pairwise distances for a small sample
# Ensure all entries in cleaned_description are valid strings
df["cleaned_description"] = df["cleaned_description"].fillna("").astype(str)

# Updated loop to skip empty strings
#n = len(df)
n=20
distance_results = []
for i in tqdm(range(n)):
    for j in range(i + 1, n):
        s1, s2 = df["cleaned_description"][i], df["cleaned_description"][j]
        
        # Skip empty or invalid strings
        if not s1.strip() or not s2.strip():
            continue

        # Calculate distances
        edit_dist = compute_edit_distance(s1, s2)
        jaro_dist = compute_jaro_distance(s1, s2)
        jaccard_dist = compute_jaccard_distance(s1, s2)

        # Append results
        distance_results.append({
            "text1": s1,
            "text2": s2,
            "edit_distance": edit_dist,
            "jaro_distance": jaro_dist,
            "jaccard_distance": jaccard_dist
        })

# Convert results to DataFrame
distances_df = pd.DataFrame(distance_results)



100%|██████████| 20/20 [00:00<00:00, 3067.47it/s]


In [8]:
from collections import defaultdict

def group_similar_descriptions(df, threshold=0.8):
    """Group descriptions based on similarity."""
    texts = df["cleaned_description"].tolist()
    tfidf_sim_matrix = compute_tfidf_similarity(texts)
    groups = defaultdict(set)  # Use set to avoid duplicates

    for i, text in enumerate(texts):
        for j, score in enumerate(tfidf_sim_matrix[i]):
            if score > threshold and i != j:
                groups[text].add(df["description"].iloc[j])
                groups[df["description"].iloc[j]].add(text)  # Bidirectional grouping

    # Convert sets to lists
    for key in groups:
        groups[key] = list(groups[key])

    return groups

# Call the function
similar_groups = group_similar_descriptions(df, threshold=0.75)

# Display Similar Groups
for key, group in similar_groups.items():
    print(f"\nGroup Leader: {key}")
    print("Similar Descriptions:")
    for g in group:
        print(f"  - {g}")

: 

: 

In [8]:
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix



def group_similar_descriptions(df, threshold=0.75, batch_size=500):
    """
    Group descriptions based on similarity with memory optimization.
    Processes TF-IDF similarity in batches.
    """
    texts = df["cleaned_description"].tolist()
    descriptions = df["Description"].tolist()
    tfidf_matrix = compute_tfidf_similarity(texts)

    # Initialize groups
    groups = defaultdict(set)

    # Batch processing to compute cosine similarity
    n = len(texts)
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        batch_matrix = tfidf_matrix[start:end]

        # Compute cosine similarity for the batch
        sim_matrix = cosine_similarity(batch_matrix, tfidf_matrix)

        # Process similarity scores
        for i in range(sim_matrix.shape[0]):
            for j in range(sim_matrix.shape[1]):
                if sim_matrix[i, j] > threshold and start + i != j:
                    groups[texts[start + i]].add(descriptions[j])
                    groups[descriptions[j]].add(texts[start + i])  # Bidirectional grouping

    # Convert sets to lists
    for key in groups:
        groups[key] = list(groups[key])

    return groups

# Example usage
# Assuming df is your DataFrame and contains the 'cleaned_description' column
similar_groups = group_similar_descriptions(df, threshold=0.75, batch_size=200)

# Display Similar Groups
for key, group in list(similar_groups.items())[:5]:  # Display first 5 groups for brevity
    print(f"\nGroup Leader: {key}")
    print("Similar Descriptions:")
    for g in group:
        print(f"  - {g}")


Group Leader: ground fl facade to erect illuminated electric wall single faced sign na for maher ordinance
Similar Descriptions:
  - to erect electric, wall, single faced sign. (illuminated)** maher: n/a **
  - to erect electric, wall, single faced sign.** maher: n/a **
  - to erect electric, wall, single faced sign. ** maher: n/a **
  - to erect an illuminated, electric, single faced wall sign logo. ** maher: compliance with ordinance no# 155-13 not required **
  - #6:  erect an electric single faced wall sign; maher n/a
  - 3 illuminated electric single faced wall sign.
  - #144 - erect an electric single faced wall sign. n/a maher ordinance
  - erect one single faced electric wall sign. n/a ordinance #155-13
  - erect an electric single faced wall sign. n/a maher ordinance
  - erect one illuminated single faced sign (v20);  maher n/a
  - erect illuminated wall single faced sign; maher n/a
  - erect (1) single faced electric wall sign. n/a ordinance #155-13
  - erect 1 illuminated e